In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
batch_size = 8
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(10).batch(batch_size)

In [ ]:
print(next(iter(dataset)))

(<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>, <tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>)


In [ ]:
#批次處理（Batching）
#tf.distribute.MirroredStrategy
batch_size = 8
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(10).batch(batch_size)
#透過experimental_distribute_dataset分開資料 
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
print(next(iter(dist_dataset)))

(<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>, <tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>)


In [ ]:
#單一輸入
dataset = tf.data.Dataset.range(8).batch(batch_size)
#透過experimental_distribute_dataset分開資料 
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
print(next(iter(dist_dataset)))

tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)


In [ ]:
#分散式輸入
batch_size = 4
steps_per_loop = 5
strategy = tf.distribute.MirroredStrategy(devices=["GPU:0", "CPU:0"])

dataset = tf.data.Dataset.range(8).batch(batch_size)
distributed_iterator = iter(strategy.experimental_distribute_dataset(dataset))

@tf.function
def train_fn(distributed_iterator):
  for _ in tf.range(steps_per_loop):
    optional_data = distributed_iterator.get_next_as_optional()
    if not optional_data.has_value():
      break
    per_replica_results = strategy.run(lambda x:x, args=(optional_data.get_value(),))
    tf.print(strategy.experimental_local_results(per_replica_results))
train_fn(distributed_iterator)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:CPU:0')
([0 1], [2 3])
([4 5], [6 7])
